# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = pd.read_csv('../output_data/weather_df.csv')

weather_df = file
del weather_df["Unnamed: 0"]
weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hobart,-42.8794,147.3294,68.00,45,75,16.11,AU,1612661779
1,Punta Arenas,-53.1500,-70.9167,51.80,66,0,17.27,CL,1612661595
2,George Town,5.4112,100.3354,80.60,83,20,2.30,MY,1612661728
3,Puerto Ayora,-0.7393,-90.3518,77.00,93,50,6.69,EC,1612661537
4,Carnarvon,-24.8667,113.6333,84.20,100,40,13.80,AU,1612661784
...,...,...,...,...,...,...,...,...,...
577,Royan,45.6285,-1.0281,42.80,87,0,8.05,FR,1612662604
578,Koslan,63.4564,48.8989,-22.88,83,65,10.87,RU,1612662605
579,Mezhdurechensk,53.6942,88.0603,21.02,95,37,3.96,RU,1612662445
580,Saryg-Sep,51.5000,95.6000,2.44,90,0,3.49,RU,1612662608


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

locations = weather_df[["Lat", "Lng"]]
humidity_weight = weather_df["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weight, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [5]:
narrow_weather_df = weather_df.loc[(weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70) & 
                                    (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0), :]
narrow_weather_df = narrow_weather_df.dropna(how='any')
narrow_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
125,Iquique,-20.2208,-70.1431,73.40,49,0,9.22,CL,1612661958
135,Yangjiang,21.8500,111.9667,78.01,45,0,3.49,CN,1612661972
152,Assaí,-23.3733,-50.8414,75.20,57,0,5.75,BR,1612661996
170,Vao,-22.6667,167.4833,78.03,80,0,5.46,NC,1612662023
174,Lázaro Cárdenas,17.9583,-102.2000,73.83,75,0,1.74,MX,1612662029
176,Formosa,-26.1775,-58.1781,71.60,78,0,5.75,AR,1612662032
249,Crateús,-5.1783,-40.6775,76.32,57,0,3.04,BR,1612662136
274,Puerto Baquerizo Moreno,-0.9000,-89.6000,76.73,85,0,7.78,EC,1612662138
278,Ban Phai,16.0600,102.7310,78.01,50,0,7.40,TH,1612662177
283,Te Anau,-45.4167,167.7167,75.90,38,0,5.68,NZ,1612662184


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrow_weather_df

# Add Hotel Name column to hotel_df        
hotel_df["Hotel Name"] = ""

for index, row in hotel_df.iterrows():

    #base url
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #Setting parameters with dictionary to search for hotels
    params = {"location": str(row["Lat"]) + "," +str(row["Lng"]),
              "radius": str(5000),
              "type": "Hotel",
              "keyword": "Hotel",
              "key": g_key}
    
    response = requests.get(url, params=params).json()
  
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
        

    except (KeyError, IndexError):
        print("Missing")    


Missing
Missing


In [7]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
125,Iquique,-20.2208,-70.1431,73.40,49,0,9.22,CL,1612661958,Holiday Inn Express Iquique
135,Yangjiang,21.8500,111.9667,78.01,45,0,3.49,CN,1612661972,Yangjiang Country Garden Phoenix Hotel （West G...
152,Assaí,-23.3733,-50.8414,75.20,57,0,5.75,BR,1612661996,Hotel Sol Nascente
170,Vao,-22.6667,167.4833,78.03,80,0,5.46,NC,1612662023,OURE TERA beach resort
174,Lázaro Cárdenas,17.9583,-102.2000,73.83,75,0,1.74,MX,1612662029,City Express Lázaro Cárdenas
176,Formosa,-26.1775,-58.1781,71.60,78,0,5.75,AR,1612662032,Howard Johnson Hotel And Casino Formosa
249,Crateús,-5.1783,-40.6775,76.32,57,0,3.04,BR,1612662136,HOTEL POTY
274,Puerto Baquerizo Moreno,-0.9000,-89.6000,76.73,85,0,7.78,EC,1612662138,Casa Blanca
278,Ban Phai,16.0600,102.7310,78.01,50,0,7.40,TH,1612662177,Jasper Hotel Ban Phai โรงแรมแจสเปอร์โฮเทล
283,Te Anau,-45.4167,167.7167,75.90,38,0,5.68,NZ,1612662184,Distinction Luxmore Hotel Te Anau


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))